# Fala dev! 
Nesse parte do tutorial vamos entender melhor como funciona os vector databases e vector stores.

### Existe diferença entre vector database e vector store?
Na verdade sim, apesar de serem muito parecidos, a diferença entre eles é que o vector database é um banco de dados que armazena os vetores, enquanto o vector store é um serviço que armazena os vetores.

atente-se para essa tabela comparativa:

| Aspecto           | Vector Store                                  | Vector Database                                         |
|-------------------|-----------------------------------------------|---------------------------------------------------------|
| **Objetivo**      | Armazenar vetores                             | Armazenar e realizar consultas eficientes sobre vetores |
| **Funcionalidades**| Armazenamento e recuperação básica            | Indexação vetorial, consultas de similaridade, k-NN     |
| **Integração**    | Geralmente sobre outros sistemas de armazenamento | Solução independente especializada em vetores          |
| **Escalabilidade**| Limitada a funcionalidades básicas            | Otimizado para grandes volumes de vetores e alta performance |
| **Complexidade**  | Simples                                       | Mais complexo, com suporte a operações avançadas        |
| **Exemplos de Uso**| Cenários simples de armazenamento             | Motores de busca semânticos, recomendações, NLP         |
| **Exemplos de Ferramentas** | Simples módulos de armazenamento (p.ex., camadas de memória) | Pinecone, Milvus, Weaviate                              |


#### Vector Store:
FAISS (Facebook AI Similarity Search): É uma biblioteca desenvolvida pelo Facebook para buscar em grandes conjuntos de vetores. Embora seja poderosa, FAISS é mais uma biblioteca de indexação e busca, não um banco de dados completo.


#### Vector Database:
Pinecone: Uma solução de banco de dados vetorial gerenciado que oferece armazenamento, indexação e consulta de vetores. É projetado para busca e recuperação de alta performance em vetores.


### Vamos ver um pouco mais sobre eles

Uma das maneiras mais comuns de armazenar e pesquisar dados não estruturados é incorporá-los e armazenar os vetores de incorporação resultantes e, em seguida, no momento da consulta, incorporar a consulta não estruturada e recuperar os vetores de incorporação que são 'mais semelhantes' à consulta incorporada. Um armazenamento de vetores se encarrega de armazenar dados incorporados e realizar pesquisas de vetores para você.

<img src="https://python.langchain.com/v0.1/assets/images/vector_stores-125d1675d58cfb46ce9054c9019fea72.jpg" alt="vectorDB">


Este passo a passo apresenta funcionalidades básicas relacionadas aos armazenamentos de vetores. Uma parte fundamental do trabalho com armazenamentos de vetores é criar o vetor a ser colocado neles, que geralmente é criado por meio de embeddings. Portanto, é recomendável que você se familiarize com as interfaces do modelo de incorporação de texto antes de mergulhar nisso.

Existem muitas opções excelentes de armazenamento de vetores. Aqui estão algumas que são gratuitas, de código aberto e executadas inteiramente em sua máquina local. Revise todas as integrações para muitas ofertas hospedadas excelentes.

Este passo a passo usa o banco de dados de vetores FAISS, que faz uso da biblioteca Facebook AI Similarity Search (FAISS).

In [23]:
import os
from dotenv import load_dotenv
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS


# Uma forma simples de carregar o documento
raw_documents = TextLoader('./state_of_the_union.txt', encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

#### Deu errado aí mano?
Olhou o que precisa fazer ali no terminal ?
Tu esqueceu de novo de dar pip install numa altura dessa do campeonato? 

mas é cpu ou gpu?
Se não souber a diferença entre cpu e gpu, dá uma olhada nesse [link](https://www.techtudo.com.br/dicas-e-tutoriais/2020/06/cpu-vs-gpu-entenda-a-diferenca-entre-os-processadores.ghtml)

### Busca por similaridade
Aqui vamos ver como fazer uma busca por similaridade em um banco de dados de vetores.

In [14]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


Também é possível fazer uma busca por documentos semelhantes a um determinado vetor de incorporação usando similarity_search_by_vector que aceita um vetor de incorporação como parâmetro em vez de uma string

In [15]:
embedding_vector = OpenAIEmbeddings().embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


### Interessante não é mesmo ?
Você ja está guardando os dados em um banco de dados de vetores e agora pode fazer buscas por similaridade.

Vamos ver outros tipos de vectors databases.

In [17]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# Load the document, split it into chunks, and embed each chunk.
loader = TextLoader("state_of_the_union.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()


Em seguida, vá para o console [Pinecone](https://login.pinecone.io/login?state=hKFo2SAyYVlyVFR5TFZRQW81T2tjel85YUdxeDJLQTFhbzI4bqFupWxvZ2luo3RpZNkgQUpfVEtwOGNwTXhaVnZ1dWtlOGUzaDJocXYyNGFqcmujY2lk2SBUOEkyaEc2Q2FaazUwT05McWhmN3h6a1I0WmhMcVM0Qw&client=T8I2hG6CaZk50ONLqhf7xzkR4ZhLqS4C&protocol=oauth2&audience=https%3A%2F%2Fus-central1-production-console.cloudfunctions.net%2Fapi%2Fv1&scope=openid%20profile%20email%20read%3Acurrent_user&redirect_uri=https%3A%2F%2Fapp.pinecone.io&sessionType=signup&response_type=code&response_mode=query&nonce=dHBBbmlySldKN3BTekpmUjRJOWtlcGpHaVBhVHBoRzRJMTAwNEM2LkdURA%3D%3D&code_challenge=XM-Ot2SDprYNy1UYJrGqsj4GHd9j-kTjT2ht8lPEUF8&code_challenge_method=S256&auth0Client=eyJuYW1lIjoiYXV0aDAtcmVhY3QiLCJ2ZXJzaW9uIjoiMS4xMi4xIn0%3D) e crie um novo índice com dimensão=1536 chamado "langchain-test-index". Em seguida, copie a chave API e o nome do índice.

In [27]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec


load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
index_name = "langchain-test-index"

pc = Pinecone(api_key=PINECONE_API_KEY)
# Connect ao Pinecone e carrega os documentos
docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

Busca por similaridade

In [28]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


### Pesquisa de similaridade por vetor

​Também é possível fazer uma busca por documentos semelhantes a um determinado vetor de incorporação usando similarity_search_by_vector que aceita um vetor de incorporação como parâmetro em vez de uma string

In [29]:
embedding_vector = OpenAIEmbeddings().embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


### RAG e Vector Databases
Bem agora é impossível entender o conceito de vector databases sem falar sobre o RAG, que é um modelo de linguagem que tem sido muito utilizado para gerar textos e respostas em linguagem natural.

O RAG é um modelo de linguagem que combina um modelo de geração de linguagem com um mecanismo de pesquisa. O modelo de geração de linguagem é treinado para gerar texto, enquanto o mecanismo de pesquisa é treinado para recuperar informações relevantes de um banco de dados de vetores.

Para entender uma aplição de RAG simples, vamos disponibilizar esse repositório [aqui](https://github.com/xNero-AI/RAG_and_RS) como exemplo.

### "A por que tá muito tranquilo e quero mais coisas pra estudar..."
Segue alguns links para você se aprofudnar mais no assunto:
- [FAISS](https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/)

- [Pinecone](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)

- [Weaviate](https://python.langchain.com/v0.2/docs/integrations/vectorstores/weaviate/)

- [Milvus](https://python.langchain.com/v0.2/docs/integrations/vectorstores/milvus/)

- [Vector Database](https://python.langchain.com/v0.2/docs/integrations/vectorstores/vector-database/)

- [Vector Store](https://python.langchain.com/v0.2/docs/integrations/vectorstores/vector-store/)
